<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Pre-Processed-Datasets-NLP/diabeticretinopathyPPFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
import re

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/DSGP PROJECT 29/DATASETS/Synthetic_Diabetic_Retinopathy_Dataset.csv')

# Step 1: Handle Missing Values
# Check for missing values and replace them with the most frequent category
imputer = SimpleImputer(strategy='most_frequent')
df.iloc[:, :] = imputer.fit_transform(df)

# Step 2: Feature Engineering
# Creating a new feature as an example: Total Cholesterol to HDL ratio
df['Cholesterol_HDL_Ratio'] = df['Total Cholesterol'] / df['HDL Cholesterol']

# Step 3: Text Processing
# Simple text processing using basic Python functions and regex
def clean_text(text):
    text = text.lower()  # convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    tokens = text.split()  # tokenize the text by splitting
    return ' '.join(tokens)  # join tokens back into a string

df['Visual Symptoms'] = df['Visual Symptoms'].apply(clean_text)

# Step 4: Encode Categorical Data
# Convert binary categorical data using OneHotEncoder
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(df[['Smoking Status']])
encoded_feature_names = encoder.get_feature_names(['Smoking Status'])
df[encoded_feature_names] = encoded_features
df.drop(['Smoking Status'], axis=1, inplace=True)

# Step 5: Normalize/Scale Numerical Data
# Scale numerical features to have zero mean and unit variance
scaler = StandardScaler()
numerical_features = ['Age', 'Retinal Thickness', 'Cotton Wool Spots Count', 'LDL Cholesterol',
                      'HDL Cholesterol', 'Total Cholesterol', 'Microaneurysms Count',
                      'Hemorrhages Count', 'Cholesterol_HDL_Ratio']
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Step 6: Handle Class Imbalance
# Use SMOTE to handle class imbalance
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis'].astype(int)
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Combine the resampled features and target into a new DataFrame
df_resampled = pd.DataFrame(X_res, columns=X.columns)
df_resampled['Diagnosis'] = y_res

# Step 7: Save Preprocessed Data
# Save the final preprocessed data to a new CSV file
final_file_path = '/content/drive/MyDrive/DSGP PROJECT 29/FINAL MODEL/Preprocessed_Diabetic_Retinopathy_Dataset.csv'
df_resampled.to_csv(final_file_path, index=False)

final_file_path


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'